In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_excel('./dataset20220613_label.xlsx')
df.shape

(1528, 90)

In [2]:
df.sample(5)

,Filnum,IDps,condition,Y,ltV,leV,trV,leS,trS,leT,...,"trP(1,:)",Unnamed: 81,Unnamed: 82,"trP(2,:)",Unnamed: 84,Unnamed: 85,"trP(3,:)",Unnamed: 87,Unnamed: 88,label
1300,82,0.177670,11,1,-3.233881,9.766703,9.354274,1250.1380,1262.827,128,...,-0.001303,-0.007711,-0.001229,0.000419,0.001279,0.003634,0.000136,0.000280,0.000010,2
238,9,1.001995,2,2,-3.920796,12.107970,11.813343,1210.7970,1204.961,100,...,-0.000143,0.011415,-0.008325,0.001948,0.020780,-0.014275,0.000043,0.000339,0.000953,0
663,42,0.696900,6,1,-3.734410,11.380487,10.120197,1308.7560,1285.265,115,...,0.000341,-0.003046,0.001187,0.001183,-0.000402,-0.001932,0.000081,0.001162,-0.000056,1
428,30,0.974528,4,1,-3.820750,11.556925,11.496944,1225.0340,1241.670,106,...,0.000645,0.000189,0.001490,0.001204,0.006240,-0.010194,0.000012,0.000066,0.001586,0
1489,90,0.619236,12,2,-3.136744,9.832269,8.211708,1317.5241,1264.603,134,...,-0.000705,0.000140,-0.000316,0.000164,0.001057,0.000333,0.000140,0.000156,0.000622,1


In [3]:
label = df['label']

In [4]:

print(label.shape)

(1528,)


## 特征工程

观察类别变量’conditon和Y是否每个类别的样本数均衡。

In [5]:
X=df.copy()

In [6]:
X['Y'].value_counts()

1    889
2    639
Name: Y, dtype: int64

In [7]:
X['condition'].value_counts()

3     138
1     136
6     135
4     131
8     128
2     127
10    127
9     126
12    126
5     122
11    119
7     113
Name: condition, dtype: int64

观察结果没有样本不均衡现象

对condition变量进行onehot编码

In [8]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
onehotarray = encoder.fit_transform(X[['condition']]).toarray()
items = [f'condition_{item}' for item in encoder.categories_[0]]
X[items] = onehotarray

In [9]:
X = X.iloc[:, 3:]

In [10]:
X.sample(5)

,Y,ltV,leV,trV,leS,trS,leT,trT,leToeUi,leswpht,...,condition_3,condition_4,condition_5,condition_6,condition_7,condition_8,condition_9,condition_10,condition_11,condition_12
113,2,-3.532832,11.330806,10.617903,1223.727,1199.823,108,113,68,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1266,2,-3.620100,11.384072,9.911717,1263.632,1258.788,111,127,112,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1071,1,-3.319942,10.820103,10.349312,1157.751,1293.664,107,125,58,49,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
959,1,-4.030377,11.812820,10.066048,1311.223,1258.256,111,125,63,48,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1414,1,-3.468066,11.036417,10.060778,1269.188,1267.658,115,126,68,47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
X.shape

(1528, 99)

类别变量Y的二分类用0和1表示

In [12]:
X['Y'] = X['Y'] - 1

把数值变量画出分布图，观察是否有异常值

In [13]:
# X.iloc[:, 1:86]
anomalous_features = X.iloc[:, 1:86].columns

In [14]:

import seaborn as sns
for i, feature in enumerate(anomalous_features):
    plt.figure(figsize=(10, 5))
    sns.distplot(X[feature],kde=True,bins=100)
    plt.savefig(f'./pics/{i}.png')
    plt.close()

d:\anaconda\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
d:\anaconda\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
d:\anaconda\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings

去除异常值

In [15]:
for i, feature in enumerate(anomalous_features):
    upper_lim = X[feature].quantile(.95)
    lower_lim = X[feature].quantile(.05)
    X_no_outlier = X[(X[feature] < upper_lim) & (X[feature] > lower_lim)]

In [16]:
X_no_outlier.shape

(1374, 99)

对数值列，每一列标准化

In [17]:
from sklearn.preprocessing import StandardScaler
X_number_data = StandardScaler().fit_transform(X_no_outlier[anomalous_features])
X_number_data = pd.DataFrame(data = X_number_data, columns = anomalous_features)
X_number_data.describe()

,ltV,leV,trV,leS,trS,leT,trT,leToeUi,leswpht,trToeUi,...,Unnamed: 79,"trP(1,:)",Unnamed: 81,Unnamed: 82,"trP(2,:)",Unnamed: 84,Unnamed: 85,"trP(3,:)",Unnamed: 87,Unnamed: 88
count,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,...,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03,1.374000e+03
mean,-1.118707e-15,-1.253496e-15,2.807070e-16,2.020622e-15,-5.953106e-16,3.949614e-16,1.376870e-16,-1.006796e-16,-2.165501e-17,-3.317741e-16,...,4.813795e-16,-8.807446e-18,1.616045e-17,-1.850372e-17,-2.293168e-16,1.912589e-16,7.267355e-16,9.547797e-17,2.262463e-18,-2.537999e-16
std,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,...,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00,1.000364e+00
min,-3.196905e+00,-9.216194e+00,-7.892316e+00,-2.226518e+01,-2.382233e+01,-5.524272e+00,-3.772851e+00,-1.820098e+00,-1.408786e+00,-1.435836e+00,...,-4.165445e+00,-4.246588e+00,-8.887724e+00,-7.536225e+00,-1.680528e+01,-5.167992e+00,-2.594620e+00,-3.678233e+00,-6.529348e+00,-1.177382e+00
25%,-6.158258e-01,-6.325531e-01,-7.335208e-01,-4.728949e-01,-4.987244e-01,-6.429527e-01,-8.845794e-01,-1.068627e-02,-4.327622e-01,-1.294024e+00,...,-4.337320e-01,-4.879447e-01,-3.001965e-01,-4.100449e-01,-3.191092e-01,-5.957521e-01,-7.723863e-01,-7.113356e-01,-4.546567e-01,-7.026820e-01
50%,4.996753e-03,-5.878874e-02,-1.154290e-01,-2.834048e-02,-7.581202e-02,-3.278785e-02,1.469461e-01,1.376262e-01,-1.345328e-01,4.644515e-01,...,4.724191e-01,1.329642e-01,1.321817e-01,3.920818e-02,-5.835945e-02,-2.103288e-01,1.744081e-01,1.470770e-01,9.179501e-02,-4.715550e-01
75%,6.493003e-01,5.854538e-01,7.298639e-01,4.314746e-01,5.404890e-01,5.773770e-01,6.970930e-01,3.156011e-01,1.365849e-01,8.260734e-01,...,7.077246e-01,6.386727e-01,4.753390e-01,3.724151e-01,3.810707e-01,4.428320e-01,4.806938e-01,6.896962e-01,5.734278e-01,5.886193e-01
max,1.821330e+01,3.300111e+00,5.651343e+00,2.599192e+00,2.467943e+00,3.221425e+00,2.897681e+00,2.362313e+00,2.413973e+00,1.684039e+00,...,1.074344e+00,4.671029e+00,4.270606e+00,7.936889e+00,5.697775e+00,4.971944e+00,3.950922e+00,5.740930e+00,5.818230e+00,2.785805e+00


In [18]:
X_number_data.iloc[:,13]

0       0.687397
1      -1.138647
2      -1.158541
3       0.953059
4      -0.321234
          ...   
1369   -1.162064
1370    0.794848
1371   -0.077295
1372   -0.473334
1373   -0.907777
Name: pelv(1,:), Length: 1374, dtype: float64

我的想法是24个xyz坐标，总共72列，在探究敏感性的时候想知道的是哪个坐标系起的作用，而不是坐标系的xyz起的作用，所以对xyz三列pca成一列

In [19]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X_useful =  X_number_data.iloc[:,:13]
X_coord = X_number_data.iloc[:,13:]
coord_names = anomalous_features[13:]
for i in range(72//3):
    tmp = X_coord.iloc[:,i*3:(i+1)*3]
    X_useful[coord_names[i*3]] = pca.fit_transform(tmp)

C:\Users\admin\AppData\Local\Temp/ipykernel_8284/2638398023.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_useful[coord_names[i*3]] = pca.fit_transform(tmp)
C:\Users\admin\AppData\Local\Temp/ipykernel_8284/2638398023.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_useful[coord_names[i*3]] = pca.fit_transform(tmp)
C:\Users\admin\AppData\Local\Temp/ipykernel_8284/2638398023.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [20]:
X_useful.shape
#X_useful是处理好的数值数据，接下来加上刚刚的Y和condition类别数据

(1374, 37)

In [21]:
X_useful = X_useful.reset_index(drop=True)
X_no_outlier = X_no_outlier.reset_index(drop=True)
X_final = pd.concat([X_useful, X_no_outlier['Y'], X_no_outlier.iloc[:,87:],X_no_outlier['label']], axis=1, ignore_index=True)


In [22]:
X_final.shape

(1374, 51)

In [23]:
col_names =[*list(X_useful.columns), 'Y', *list(X_no_outlier.iloc[:,87:].columns.values), 'label']
X_ff = pd.DataFrame(X_final.values, columns=col_names)

In [24]:
X_ff.sample(5)

,ltV,leV,trV,leS,trS,leT,trT,leToeUi,leswpht,trToeUi,...,condition_4,condition_5,condition_6,condition_7,condition_8,condition_9,condition_10,condition_11,condition_12,label
158,-0.969783,1.379171,1.727528,-0.465125,-0.169023,-1.558200,-1.572263,0.048639,-0.459874,0.379364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
730,1.433686,-0.976022,-1.488038,-0.318993,-1.215664,0.882459,1.178472,0.226614,0.028138,-1.379111,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
184,-0.212807,-0.068240,0.387926,0.370451,-0.205055,0.170600,-0.540738,-1.731110,1.627732,0.577902,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
424,-1.059836,1.166744,1.292491,0.076976,-0.044068,-1.151423,-1.228421,0.048639,-0.351427,0.436089,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
978,1.352976,-1.492548,-1.394328,-1.005918,-0.558489,1.187542,1.316008,-1.731110,1.898850,-1.294024,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [25]:
X_ff.to_csv('./data_ff.csv', index=False)